# Cat vs Non-Cat Classifier

A 5-Layer Deep Neural Network for classifying cat images.

**Features:**
- Data loading and preprocessing
- Training with customizable learning rate
- Evaluation metrics (Accuracy, Precision, Recall, F1)
- Visualization of cost curves, confusion matrix, and predictions

## Import Libraries

In [ ]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
from five_layer_nn import FiveLayerNN

## Data Loading

In [ ]:
def load_data():
    """
    Load the cat vs non-cat dataset.

    Returns:
    train_x_orig -- training images, shape (num_examples, height, width, channels)
    train_y -- training labels, shape (1, num_train_examples)
    test_x_orig -- test images, shape (num_examples, height, width, channels)
    test_y -- test labels, shape (1, num_test_examples)
    classes -- list of class names
    """
    # Load training data
    train_dataset = h5py.File('datasets/train_catvnoncat.h5', "r")
    train_x_orig = np.array(train_dataset["train_set_x"][:])
    train_y = np.array(train_dataset["train_set_y"][:])

    # Load test data
    test_dataset = h5py.File('datasets/test_catvnoncat.h5', "r")
    test_x_orig = np.array(test_dataset["test_set_x"][:])
    test_y = np.array(test_dataset["test_set_y"][:])

    # Get class names
    classes = np.array(test_dataset["list_classes"][:])

    # Reshape labels to (1, num_examples)
    train_y = train_y.reshape((1, train_y.shape[0]))
    test_y = test_y.reshape((1, test_y.shape[0]))

    return train_x_orig, train_y, test_x_orig, test_y, classes

In [ ]:
def preprocess_data(train_x_orig, test_x_orig):
    """
    Preprocess images: flatten and normalize.

    Arguments:
    train_x_orig -- training images, shape (num_examples, height, width, channels)
    test_x_orig -- test images, shape (num_examples, height, width, channels)

    Returns:
    train_x -- flattened and normalized training images, shape (num_features, num_examples)
    test_x -- flattened and normalized test images, shape (num_features, num_examples)
    """
    # Flatten images: (num_examples, h, w, c) -> (h*w*c, num_examples)
    train_x_flatten = train_x_orig.reshape(train_x_orig.shape[0], -1).T
    test_x_flatten = test_x_orig.reshape(test_x_orig.shape[0], -1).T

    # Normalize pixel values (0-255 -> 0-1)
    train_x = train_x_flatten / 255.
    test_x = test_x_flatten / 255.

    return train_x, test_x

## Evaluation Metrics

In [ ]:
def compute_metrics(predictions, Y):
    """
    Compute precision, recall, F1 score, and confusion matrix.

    Arguments:
    predictions -- predicted labels (0 or 1), shape (1, m)
    Y -- true labels (0 or 1), shape (1, m)

    Returns:
    metrics -- dictionary with precision, recall, f1, confusion matrix
    """
    predictions = predictions.flatten()
    Y = Y.flatten()

    # True Positives, False Positives, True Negatives, False Negatives
    TP = np.sum((predictions == 1) & (Y == 1))
    FP = np.sum((predictions == 1) & (Y == 0))
    TN = np.sum((predictions == 0) & (Y == 0))
    FN = np.sum((predictions == 0) & (Y == 1))

    # Precision: Of all predicted cats, how many are actually cats?
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0

    # Recall: Of all actual cats, how many did we correctly identify?
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0

    # F1 Score: Harmonic mean of precision and recall
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    # Accuracy
    accuracy = (TP + TN) / (TP + TN + FP + FN)

    return {
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'accuracy': accuracy,
        'TP': TP,
        'FP': FP,
        'TN': TN,
        'FN': FN
    }

## Plotting Functions

In [ ]:
def plot_cost(losses, title="Cost over Training"):
    """Plot the training cost curve."""
    plt.figure(figsize=(10, 6))
    plt.plot(losses, 'b-', linewidth=2)
    plt.xlabel('Iterations (per 100)', fontsize=12)
    plt.ylabel('Cost', fontsize=12)
    plt.title(title, fontsize=14)
    plt.grid(True, alpha=0.3)
    plt.savefig('cost_curve.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("    Saved: cost_curve.png")

In [ ]:
def plot_learning_rate_comparison(train_x, train_y, layer_dims, learning_rates, epochs=2500):
    """
    Train models with different learning rates and compare.

    Arguments:
    train_x -- training data
    train_y -- training labels
    layer_dims -- network architecture
    learning_rates -- list of learning rates to try
    epochs -- number of training epochs
    """
    plt.figure(figsize=(12, 8))

    results = {}

    for lr in learning_rates:
        print(f"\n    Training with learning_rate = {lr}...")
        nn = FiveLayerNN(layer_dims, learning_rate=lr)
        losses = nn.train(train_x, train_y, epochs=epochs, print_loss=False)

        # Sample losses every 100 iterations for plotting
        sampled_losses = losses[::1]  # Already sampled in train()

        results[lr] = {
            'losses': sampled_losses,
            'final_loss': losses[-1],
            'model': nn
        }

        plt.plot(sampled_losses, label=f'lr = {lr}', linewidth=2)

    plt.xlabel('Iterations (per 100)', fontsize=12)
    plt.ylabel('Cost', fontsize=12)
    plt.title('Learning Rate Comparison', fontsize=14)
    plt.legend(fontsize=10)
    plt.grid(True, alpha=0.3)
    plt.savefig('learning_rate_comparison.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("\n    Saved: learning_rate_comparison.png")

    return results

In [ ]:
def plot_confusion_matrix(metrics, title="Confusion Matrix"):
    """Plot confusion matrix as a heatmap."""
    cm = np.array([[metrics['TN'], metrics['FP']],
                   [metrics['FN'], metrics['TP']]])

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title(title, fontsize=14)
    plt.colorbar()

    classes = ['Non-Cat', 'Cat']
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, fontsize=12)
    plt.yticks(tick_marks, classes, fontsize=12)

    # Add text annotations
    thresh = cm.max() / 2.
    for i in range(2):
        for j in range(2):
            plt.text(j, i, format(cm[i, j], 'd'),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black",
                    fontsize=16)

    plt.ylabel('Actual', fontsize=12)
    plt.xlabel('Predicted', fontsize=12)
    plt.tight_layout()
    plt.savefig('confusion_matrix.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("    Saved: confusion_matrix.png")

In [ ]:
def plot_sample_predictions(test_x_orig, test_y, predictions, classes, num_samples=10):
    """Plot sample images with predictions."""
    fig, axes = plt.subplots(2, 5, figsize=(15, 6))
    axes = axes.flatten()

    indices = np.random.choice(test_x_orig.shape[0], num_samples, replace=False)

    for idx, ax in enumerate(axes):
        i = indices[idx]
        ax.imshow(test_x_orig[i])

        pred_label = classes[int(predictions[0, i])].decode("utf-8")
        true_label = classes[int(test_y[0, i])].decode("utf-8")

        color = 'green' if predictions[0, i] == test_y[0, i] else 'red'
        ax.set_title(f'Pred: {pred_label}\nTrue: {true_label}', color=color, fontsize=10)
        ax.axis('off')

    plt.suptitle('Sample Predictions (Green=Correct, Red=Wrong)', fontsize=14)
    plt.tight_layout()
    plt.savefig('sample_predictions.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("    Saved: sample_predictions.png")

In [ ]:
def plot_metrics_bar(train_metrics, test_metrics):
    """Plot comparison of metrics between train and test."""
    metrics_names = ['Accuracy', 'Precision', 'Recall', 'F1 Score']
    train_values = [train_metrics['accuracy'], train_metrics['precision'],
                    train_metrics['recall'], train_metrics['f1']]
    test_values = [test_metrics['accuracy'], test_metrics['precision'],
                   test_metrics['recall'], test_metrics['f1']]

    x = np.arange(len(metrics_names))
    width = 0.35

    fig, ax = plt.subplots(figsize=(10, 6))
    bars1 = ax.bar(x - width/2, train_values, width, label='Train', color='steelblue')
    bars2 = ax.bar(x + width/2, test_values, width, label='Test', color='coral')

    ax.set_ylabel('Score', fontsize=12)
    ax.set_title('Model Performance Metrics', fontsize=14)
    ax.set_xticks(x)
    ax.set_xticklabels(metrics_names, fontsize=12)
    ax.legend(fontsize=10)
    ax.set_ylim(0, 1.1)
    ax.grid(True, alpha=0.3, axis='y')

    # Add value labels on bars
    for bar in bars1:
        height = bar.get_height()
        ax.annotate(f'{height:.2f}', xy=(bar.get_x() + bar.get_width()/2, height),
                    xytext=(0, 3), textcoords="offset points", ha='center', va='bottom')
    for bar in bars2:
        height = bar.get_height()
        ax.annotate(f'{height:.2f}', xy=(bar.get_x() + bar.get_width()/2, height),
                    xytext=(0, 3), textcoords="offset points", ha='center', va='bottom')

    plt.tight_layout()
    plt.savefig('metrics_comparison.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("    Saved: metrics_comparison.png")

## Step 1: Load Data

In [ ]:
print("=" * 60)
print("    CAT VS NON-CAT CLASSIFIER")
print("    5-Layer Deep Neural Network")
print("=" * 60)

print("\n[1] Loading data...")
try:
    train_x_orig, train_y, test_x_orig, test_y, classes = load_data()
    print(f"    Training examples: {train_x_orig.shape[0]}")
    print(f"    Test examples: {test_x_orig.shape[0]}")
    print(f"    Image shape: {train_x_orig.shape[1:]}")
    has_real_data = True

except FileNotFoundError:
    print("    Dataset not found! Using synthetic data for demo...")
    np.random.seed(1)
    n_x = 12288
    m_train = 209
    m_test = 50

    train_x = np.random.randn(n_x, m_train)
    train_y = (np.random.rand(1, m_train) > 0.5).astype(int)
    test_x = np.random.randn(n_x, m_test)
    test_y = (np.random.rand(1, m_test) > 0.5).astype(int)
    classes = np.array([b'non-cat', b'cat'])
    train_x_orig = None
    test_x_orig = None

    print(f"    Synthetic training examples: {m_train}")
    print(f"    Synthetic test examples: {m_test}")
    has_real_data = False

## Step 2: Preprocess Data

In [ ]:
print("\n[2] Preprocessing data...")

if has_real_data:
    train_x, test_x = preprocess_data(train_x_orig, test_x_orig)
    print(f"    Flattened image size: {train_x.shape[0]}")

print(f"    Train shape: {train_x.shape}")
print(f"    Test shape: {test_x.shape}")

## Step 3: Define Network Architecture

In [ ]:
print("\n[3] Setting up network architecture...")
n_x = train_x.shape[0]
layer_dims = [n_x, 20, 7, 5, 3, 1]

print(f"    Architecture: {layer_dims}")
print(f"    Layer 1: {layer_dims[0]} -> {layer_dims[1]} (ReLU)")
print(f"    Layer 2: {layer_dims[1]} -> {layer_dims[2]} (ReLU)")
print(f"    Layer 3: {layer_dims[2]} -> {layer_dims[3]} (ReLU)")
print(f"    Layer 4: {layer_dims[3]} -> {layer_dims[4]} (ReLU)")
print(f"    Layer 5: {layer_dims[4]} -> {layer_dims[5]} (Sigmoid)")

## Step 4: Create and Train Network

In [ ]:
print("\n[4] Training network...")
print("-" * 60)

nn = FiveLayerNN(layer_dims, learning_rate=0.0075)
losses = nn.train(train_x, train_y, epochs=2500, print_loss=True)

print("-" * 60)

## Step 5: Plot Cost Curve

In [ ]:
print("\n[5] Plotting cost curve...")
plot_cost(losses, title="Training Cost - Cat vs Non-Cat Classifier")

## Step 6: Evaluate Performance with Metrics

In [ ]:
print("\n[6] Computing evaluation metrics...")

train_predictions = nn.predict(train_x)
test_predictions = nn.predict(test_x)

train_metrics = compute_metrics(train_predictions, train_y)
test_metrics = compute_metrics(test_predictions, test_y)

print("\n    === TRAINING SET METRICS ===")
print(f"    Accuracy:  {train_metrics['accuracy']*100:.2f}%")
print(f"    Precision: {train_metrics['precision']*100:.2f}%")
print(f"    Recall:    {train_metrics['recall']*100:.2f}%")
print(f"    F1 Score:  {train_metrics['f1']*100:.2f}%")

print("\n    === TEST SET METRICS ===")
print(f"    Accuracy:  {test_metrics['accuracy']*100:.2f}%")
print(f"    Precision: {test_metrics['precision']*100:.2f}%")
print(f"    Recall:    {test_metrics['recall']*100:.2f}%")
print(f"    F1 Score:  {test_metrics['f1']*100:.2f}%")

## Step 7: Plot Confusion Matrix

In [ ]:
print("\n[7] Plotting confusion matrix...")
plot_confusion_matrix(test_metrics, title="Confusion Matrix (Test Set)")

## Step 8: Plot Metrics Comparison

In [ ]:
print("\n[8] Plotting metrics comparison...")
plot_metrics_bar(train_metrics, test_metrics)

## Step 9: Learning Rate Comparison

In [ ]:
print("\n[9] Comparing different learning rates...")
learning_rates = [0.001, 0.0075, 0.01, 0.05]
lr_results = plot_learning_rate_comparison(train_x, train_y, layer_dims, learning_rates, epochs=2500)

print("\n    === LEARNING RATE COMPARISON RESULTS ===")
for lr, result in lr_results.items():
    test_acc = result['model'].accuracy(test_x, test_y)
    print(f"    lr = {lr:6.4f} | Final Loss: {result['final_loss']:.6f} | Test Accuracy: {test_acc:.2f}%")

## Step 10: Sample Predictions (if real data available)

In [ ]:
if has_real_data and test_x_orig is not None:
    print("\n[10] Plotting sample predictions...")
    plot_sample_predictions(test_x_orig, test_y, test_predictions, classes)
else:
    print("\n[10] Skipping sample predictions (no real image data available)")

## Summary

In [ ]:
print("\n" + "=" * 60)
print("    TRAINING COMPLETE!")
print("=" * 60)
print("\n    Generated Graphs:")
print("    - cost_curve.png")
print("    - confusion_matrix.png")
print("    - metrics_comparison.png")
print("    - learning_rate_comparison.png")
if has_real_data:
    print("    - sample_predictions.png")
print("\n" + "=" * 60)